In [2]:
%runfile testcong.py

## Read all optimal curves from the internal database (conductors from $1$ to $400000$) and make a hash table from the $a_p$ for $40$ primes (the first $40$ over $400000$).  This takes about 47 minutes so after doing it once we save the result, and later runs will read this (which takes a few seconds: the saved file is 52M).

In [3]:
try:
    hashtab_17_40 = load('hashtab17_40')
except IOError:
    hashtab_17_40 = make_hash(17,1,400000,40)
    hashtab_17_40 = dict([(k,v) for k,v in hashtab_17_40.items() if len(v)>1])
    save(hashtab_17_40, 'hashtab17_40')
len(hashtab_17_40)

8

In [4]:
hashtab_17_40.values()

[['11025w1', '143325ej1'],
 ['3675o1', '47775da1'],
 ['3675b1', '47775b1'],
 ['11025bi1', '143325be1'],
 ['11025bm1', '143325ba1'],
 ['3675k1', '47775cq1'],
 ['11025r1', '143325et1'],
 ['3675g1', '47775bf1']]

## The hash table is a dictionary: congruent curves will have the same hash (but possibly some incongruent curves will also have the same hash, if they have the same values of $a_p\pmod{17}$ for all $40$ primes tested).  So we extract sets of mutually congruent curves by looking at the dictionary's values (which are lists of labels) of length greater than one.

In [5]:
congruent_sets = [s for s in hashtab_17_40.values() if len(s)>1]
print("The number of nontrivial congruent sets is {}".format(len(congruent_sets)))
from collections import Counter
sizes = Counter()
for s in congruent_sets:
    sizes[len(s)] += 1
print("Distribution of sizes of nontrivial congruent sets: {}".format(sizes))

The number of nontrivial congruent sets is 8
Distribution of sizes of nontrivial congruent sets: Counter({2: 8})


### Test each of these pairs to see if we have an actual congruence, using Kraus's criterion (possibly after twisting to reduce the conductors). 

In [6]:
bad_pairs = []
for s in congruent_sets:
    E1 = EllipticCurve(s[0])
    for r in s[1:]:
        E2 = EllipticCurve(r)
        res, info = test_cong(17,E1,E2, mumax=15000000, verbose=False)
        if not res:
            report(res,info,17,s[0],r)
            bad_pairs.append([s[0],r])
if bad_pairs:
    print("{} pairs of curves are not actually 17-congruent: {}".format(len(bad_pairs),bad_pairs))
else:
    print("all {} sets of curves really are 17-congruent".format(len(congruent_sets)))

all 8 sets of curves really are 17-congruent


### Now we reduce to a smaller set of pairs such that every congruent pair is a twist of one in the smaller set

In [7]:
j_pairs = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in congruent_sets]
j_sets = list(Set(j_pairs))
print("Out of {} pairs, there are only {} distinct pairs of j-invariants".format(len(j_pairs),len(j_sets)))

Out of 8 pairs, there are only 1 distinct pairs of j-invariants


In [8]:
congruent_pairs = []
for jset in j_sets:
    for s2 in congruent_sets:
        if jset==Set([EllipticCurve(s).j_invariant() for s in s2]):
            congruent_pairs.append(s2)
            break
print("{} congruent pairs up to twist: {}".format(len(congruent_pairs),congruent_pairs))

1 congruent pairs up to twist: [['11025w1', '143325ej1']]


In [9]:
out = open("mod17pairs.m",'w')
out.write('pairs := [\\\n')
for s in congruent_sets[:-1]:
    out.write('["{}", "{}"],\\\n'.format(s[0],s[1]))
s = congruent_sets[-1]
out.write('["{}", "{}"]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()


In [10]:
j_sets

[{-46585/243, 48412981936758748562855/77853743274432041397}]